Makes summary df from abcTau run results.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns 

import pickle
import re
from pathlib import Path
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import gaussian_kde

# add the path to the abcTau package
import sys
#sys.path.append('./abcTau')
sys.path.append('C:\\Users\\ipochino\\AppData\\Local\\anaconda3\\envs\\isttc\\Lib\\site-packages\\abcTau') # IP: replaced previous line with that; relative path was not working
import abcTau

from isttc.scripts.cfg_global import project_folder_path

In [2]:
def compute_map_1d_abctau(result):
    """
    Compute the MAP (maximum a posteriori) estimate from abcTau output (as in Zeraati et al. 2022):
      1. Smooth the posterior with Gaussian KDE
      2. Locate the maximum via grid search (take into account weights)
    """
    # Extract accepted parameter samples and weights
    theta = np.asarray(result['theta accepted']).squeeze()
    weights = np.asarray(result['weights'])
    weights = weights / np.sum(weights)  # normalize to 1

    # Fit weighted Gaussian KDE (posterior smoothing)
    kde = gaussian_kde(theta, weights=weights)

    # Create fine grid over parameter space
    grid = np.linspace(theta.min(), theta.max(), 2000)

    # Evaluate density and find peak
    density = kde(grid)
    theta_map = grid[np.argmax(density)]

    return theta_map, grid, density

In [6]:
results_folder = project_folder_path + 'results\\synthetic\\results\\param_fr_alpha_tau\\'
results_folder_abctau = project_folder_path + 'results\\synthetic\\results\\param_fr_alpha_tau_abctau\\'

abctau_runs = {'abctau_run1_ou': results_folder_abctau + 'all_abctau_run1_ou\\final_results\\',
               'abctau_run2_dst_gamma': results_folder_abctau + 'all_abctau_run2_dst_gamma\\final_results\\',
               'abctau_run3_dst_gamma': results_folder_abctau + 'all_abctau_run3_dst_gamma\\final_results\\',
               'abctau_run4_dst_gamma': results_folder_abctau + 'all_abctau_run4_dst_gamma\\final_results\\'} 

### Load data (spike trains and calculated acf's)

In [4]:
df_all = pd.read_pickle(results_folder + "summary_tau_all_long_df_all_units.pkl")
df_all.head()

,unit_id,tau,tau_lower,tau_upper,fit_r_squared,acf_decline,method,tau_ms,fr,alpha,tau_ms_true,tau_diff_abs,tau_diff_rel,ci_width,lv
0,0,2.217899,2.119224,2.316575,0.998369,True,acf_full,110.894975,7.741821,0.851099,150.923515,40.028540,26.522401,0.197350,1.307775
1,1,2.568210,2.437150,2.699270,0.997899,True,acf_full,128.410512,4.394396,0.858098,147.947233,19.536721,13.205195,0.262120,1.345006
2,2,5.227451,4.334421,6.120482,0.984960,True,acf_full,261.372568,8.587393,0.528805,267.002290,5.629722,2.108492,1.786061,1.068416
3,3,3.950209,2.250084,5.650335,0.889719,True,acf_full,197.510466,6.976707,0.300692,194.403797,3.106670,1.598050,3.400250,1.055631
4,4,3.717963,3.426456,4.009471,0.995737,True,acf_full,185.898174,0.950832,0.691458,239.986489,54.088315,22.538067,0.583014,1.357205


### Summary df

In [7]:
all_records = []

for method, path_str in abctau_runs.items():
    directory = Path(path_str)
    files = [file for file in directory.iterdir() if file.is_file()]
    print(f'{method}: N files = {len(files)}')

    for file in files:
        try:
            # extract iteration index (e.g. steps12), the last iteration is encoded in the name
            ind = file.name.find('steps')
            final_step = int(file.name[ind+5] + file.name[ind+6])

            # load abctau results
            abc_results = np.load(file, allow_pickle=True)
            theta_accepted = abc_results[final_step - 1]['theta accepted']

            # compute MAP
            theta_map, grid, density = compute_map_1d_abctau(abc_results[final_step - 1])

            # extract unit id
            match_unit_id = re.search(r"spike_train_(\d+)_", file.name)
            unit_id = int(match_unit_id.group(1)) if match_unit_id else None

            all_records.append({
                'unit_id': unit_id,
                'tau_map': theta_map,
                'method': method
            })

        except Exception as e:
            print(f'Error processing {file.name}: {e}')

abctau_all_df = pd.DataFrame(all_records)

print(f'\nFinal concatenated DataFrame shape: {abctau_all_df.shape}')


abctau_run1_ou: N files = 651
abctau_run2_dst_gamma: N files = 240
abctau_run3_dst_gamma: N files = 37
abctau_run4_dst_gamma: N files = 9

Final concatenated DataFrame shape: (937, 3)


In [ ]:
directory = Path(datasave_path)
files = [file for file in directory.iterdir() if file.is_file()]
print(f'N files {len(files)}')

records = []
for file in files:
    # print(file.name)  
    # the last iteration is encoded in the name
    ind = file.name.find('steps') 
    final_step = int(file.name[ind+5] + file.name[ind+6])
    # het results 
    abc_results = np.load(file, allow_pickle=True)
    theta_accepted = abc_results[final_step-1]['theta accepted']
    tau = theta_accepted[0]
    theta_map, grid, density = compute_map_1d_abctau(abc_results[final_step-1])

    match_unit_id = re.search(r"spike_train_(\d+)_", file.name)
    unit_id = int(match_unit_id.group(1))
    records.append({
        'unit_id': unit_id,
        'tau_mean': np.mean(tau),
        'tau_median': np.median(tau),
        'tau_map': theta_map,
        'method': 'abctau_full',
    })

tau_df = pd.DataFrame(records)

In [9]:
abctau_all_df.head()

,unit_id,tau_map,method
0,1303,282.787084,abctau_run1_ou
1,0,95.733949,abctau_run1_ou
2,1,93.162482,abctau_run1_ou
3,2,3.776298,abctau_run1_ou
4,3,6.976921,abctau_run1_ou


In [10]:
tau_df_merged = abctau_all_df.merge(df_all[df_all['method'] == "acf_full"][['unit_id', 'fr', 'alpha', 'tau_ms_true', 'lv']], 
                             how='left', on='unit_id')
tau_df_merged['tau_diff_abs'] = np.abs(tau_df_merged['tau_map'] - tau_df_merged['tau_ms_true'])
tau_df_merged['tau_diff_rel'] = tau_df_merged['tau_diff_abs'] / tau_df_merged['tau_ms_true'] * 100
tau_df_merged.sort_values(by='tau_diff_rel')

tau_df_merged.rename(columns={'tau_map': 'tau_ms'}, inplace=True)

tau_df_merged

,unit_id,tau_ms,method,fr,alpha,tau_ms_true,lv,tau_diff_abs,tau_diff_rel
0,1303,282.787084,abctau_run1_ou,2.581649,0.858982,172.572553,1.327349,110.214531,63.865620
1,0,95.733949,abctau_run1_ou,7.741821,0.851099,150.923515,1.307775,55.189566,36.567904
2,1,93.162482,abctau_run1_ou,4.394396,0.858098,147.947233,1.345006,54.784751,37.029926
3,2,3.776298,abctau_run1_ou,8.587393,0.528805,267.002290,1.068416,263.225991,98.585668
4,3,6.976921,abctau_run1_ou,6.976707,0.300692,194.403797,1.055631,187.426875,96.411119
...,...,...,...,...,...,...,...,...,...
932,4,186.406733,abctau_run4_dst_gamma,0.950832,0.691458,239.986489,1.357205,53.579755,22.326155
933,5,207.683294,abctau_run4_dst_gamma,9.756467,0.629259,147.666392,1.168037,60.016902,40.643576
934,6,93.552112,abctau_run4_dst_gamma,7.613786,0.800260,112.904124,1.317774,19.352012,17.140217
935,7,99.920098,abctau_run4_dst_gamma,7.862782,0.781671,229.046160,1.246275,129.126062,56.375563


In [14]:
tau_df_merged.to_pickle(results_folder + 'summary_tau_all_long_df_all_units_abctau.pkl')